# Python Basics Review

**Syntax**

Python reads code line by line and uses key words, characters and indenting to store data and execute functions.
Import the Python libraries used in a script at the top.  Then define global variables and functions.
Use comments to document what each section of code is doing.





# Data Types 

**Numbers** - Integer, floating point, complex numbers and Boolean

**Lists** - container for data/objects

**Tuples** - container for data/objects

**Strings** - text

**Dictionaries** - container for data/objects, like a row in a spreadsheet

**Sets** - container for data/objects

**Arcpy Feature Layer** - A temporary version of a shapefile, feature class or table that allows for spatial and attribute selections.  Any edits to the Feature Layer will be reflected in the source shapefile, feature class or table.

https://github.com/DanCranford/Workshop_Beginner_1/blob/master/Beginner_Week_1_1.ipynb

# Programming Concepts

* Creating variables and assigning data to them.  x = 1
* Data Operators, +, -, * etc...
* Comparison Operators, ==, <, >=
* Looping, for, while
* Conditional Statements, if, elif, else
* Python built in functions
* Defining custom functions


https://github.com/DanCranford/Workshop_Beginner_1/blob/master/Beginner_Week_1_2.ipynb

# Process Automation with arcpy
Let's look at an example of a typical workflow in which you may want to use python.  Here we have a series of zipped shapefiles and we'd like to clip the data that's in each of the shapefiles.  To accomplish this, we'll have to:
- list all the zipped shapefiles
- extract each zipped shapefile
- buffer an Area of Interest (AOI) shapefile
- use our buffered AOI shapefile to clip each of the other shapefiles

In [ ]:

# Script to loop through a folder of shapefiles and clip them using an input buffer
import os
import arcpy
import zipfile

# Set location for output of extracted shapefiles
example_folder = r".\Example_Data"
if not os.path.exists(example_folder):
    os.mkdir(example_folder)


extracted_folder = r".\Example_Data\Extracted_SHP"
if not os.path.exists(extracted_folder):
    os.mkdir(extracted_folder)

# Unzip all the zipped shapefiles
zips = [f for f in os.listdir('.') if '.zip' in f]
for zf in zips:
    zfile = zipfile.ZipFile(zf)
    zfile.extractall(extracted_folder)

**Bonus - List Comprehension**

List comprehensions are incredibly useful for creating and modifying lists.  It can save you all sorts of space in your code by reducing the number of lines needed to create a new list or filter an existing list.  There are also dict comprehensions!

https://www.w3schools.com/python/python_lists_comprehension.asp

In [ ]:
#Set location for output of geoprocessing tools
output_folder = r".\Example_Data\Output_Folder"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

# set input for clipping
starting_point = os.path.join(extracted_folder, 'Locations_of_Interest.shp')

# spatial join with neighborhoods
starting_point_sj = arcpy.analysis.SpatialJoin(starting_point,
                                               os.path.join(extracted_folder,'SF_Neighborhoods.shp'),
                                               os.path.join(extracted_folder,'starting_point_sj.shp')
                                              )
                                               

# create buffered shapefile to use in clip
starting_point_buffer = arcpy.Buffer_analysis(starting_point_sj, 
                                              os.path.join(output_folder, 'Location_buffer.shp'),
                                              "1 Mile")

In [ ]:
# iterate through shapefiles and create clipped shapefiles
arcpy.env.workspace = extracted_folder
shp_list = arcpy.ListFeatureClasses()
shps_to_clip = [s for s in shp_list if 'Location' not in s]

for shp in shps_to_clip:
    #Clipping all our shapefiles
    print('Intersecting', shp)
    clip_shp = os.path.join(output_folder, shp[:-4] + '_int.shp')
    arcpy.analysis.Intersect([shp, starting_point_buffer], clip_shp)

# Cursors

Arcpy has 3 types of Cursors, Search, Update and Insert.  These can be used to iterate through the table of a shapefile/feature class/feature table and access the values of each row in the table.

You also need to specify what fields to include in the Cursor.

This returns an iterator object that can be looped through like a list.

- [arcpy.da.SearchCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/searchcursor-class.htm)
- [arcpy.da.InsertCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/insertcursor-class.htm)
- [arcpy.da.UpdateCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/updatecursor-class.htm)

In [ ]:
#Example of creating a Search Cursor, note that da.SearchCursor is the newest version of this
import arcpy

shp = os.path.join(extracted_folder, "SF_Streets.shp")
flds = ['streetname', 'active', 'classcode', 'SHAPE@LENGTH']

cursor = arcpy.da.SearchCursor(shp, flds)


In [ ]:
cursor

In [ ]:
for row in cursor:
    print(type(row), row)

In [ ]:
cursor

In [ ]:
type(cursor)

In [ ]:
#Once the iterator has been iterated through, it needs to be reset before you can iterate again
for row in cursor:
    if row[1] == 0:
        print(row)

In [ ]:
#Using the reset method
cursor.reset()

In [ ]:
#Now we can iterate again!
for row in cursor:
    if row[1] == 0:
        print(row)

In [ ]:
row

In [ ]:
#The cursors are stored in the RAM memory, if you are working with large tables, you may want to delete them after you're done.
del row
del cursor

In [ ]:
cursor

In [ ]:
#Using the with keyword, we can create an object and use it.  We'll go over this more is session 2.

with arcpy.da.UpdateCursor(shp, flds) as cursor:
    for row in cursor:
        print(type(row), row)

In [ ]:
row

In [ ]:
#Update the last row to mixed case using the title string method and the updateRow Cursor method
with arcpy.da.UpdateCursor(shp, flds) as cursor:
    for row in cursor:
        if row[0] == 'IRONWOOD WAY':
            row[0] = row[0].title()
            cursor.updateRow(row)

In [ ]:
row

In [ ]:
#We can use a search cursor to focus analysis on a feature by feature basis

loc_int_layer = arcpy.MakeFeatureLayer_management(starting_point, "loc_int_layer1")

with arcpy.da.SearchCursor(starting_point, '*') as point_cursor:
    for row in point_cursor:
        query = "Location = '{}'".format(row[2])
        print(query)
        arcpy.SelectLayerByAttribute_management(loc_int_layer, "NEW_SELECTION", query)
        arcpy.FeatureClassToFeatureClass_conversion(loc_int_layer, output_folder, row[2] + '.shp')

# In Memory Workspace

The in memory workspace allows you to create temporary spatial layers that exist in the RAM memory.  This can make scripts run faster and also avoid creating shapefiles/feature classes for every step of a script.  Once the script or Python session ends, the layers will no longer exist.

Not all tools will work with the in memory workspace, so you may need to adjust your methods in some cases.  It's also not possible to create folders within the in memory workspace.

"in_memory" is the temporary workspace for ArcMap, "memory" is the new version used in ArcPro, keep this in mind if you are developing tools people will use with ArcMap.  The "in_memory" workspace also doesn't support subtypes or domains.  

These temporary workspaces are great for storing intermediary layers, but if you are working with large datasets, you may want to delete them once they are no longer needed to free up space in your RAM.

Example of format for creating a temporary buffer feature

**ArcPro** - r"memory\Buffer"

**ArcMap** - r"in_memory\Buffer"


In [ ]:
#Let's use the memory workspace to save a temporary layer in some geoprocessing
#We can use the over write output setting to overwrite the temporary layer

import os
sf_business = os.path.join(extracted_folder, "SF_Businesses.shp")

arcpy.env.overwriteOutput = True

with arcpy.da.SearchCursor(starting_point, 'Location') as point_cursor:
    for row in point_cursor:
        query = "Location = '{}'".format(row[0])
        print(query)
        arcpy.SelectLayerByAttribute_management(loc_int_layer, "NEW_SELECTION", query)
        arcpy.Buffer_analysis(loc_int_layer, r"memory\Buffer", "1 Mile")
        arcpy.Clip_analysis(sf_business, r"memory\Buffer", os.path.join(output_folder, row[0] + '_Businesses.shp'))
      

In [ ]:
#We can delete the temporary layer when we don't need it anymore
#Only really needed if you are using large datasets and will continue in your Python session/script
arcpy.Delete_management(r"memory\Buffer")    